RNN


In [2]:

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
import numpy as np


In [7]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from tensorflow.keras.utils import to_categorical
import re 



[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [8]:
nltk.download('wordnet')
nltk.download("punkt")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [20]:
data=pd.read_csv("/content/drive/MyDrive/chatbot/emotions.txt",header=None,sep=";",names=["Text","Sentiments"])

,Text,Sentiments
0,im feeling rather rotten so im not very ambiti...,sadness
1,im updating my blog because i feel shitty,sadness
2,i never make her separate from me because i do...,sadness
3,i left with my bouquet of red and yellow tulip...,joy
4,i was feeling a little vain when i did this one,sadness
...,...,...
980,im not sure how i feel more than anything im k...,joy
981,i also feel lethargic and again,sadness
982,im feeling somewhat nostalgic about the game j...,love
983,i don t feel like i should be punished to carr...,sadness


In [75]:
data[data['Sentiments'].isnull()].index.tolist()


In [77]:
data[data['Sentiments'].isna()]

,Text,Sentiments
984,i was feeling like i said humour gets me through,NaN


In [78]:
data['Sentiments']=data['Sentiments'].fillna("joy")

In [96]:
uni=data['Sentiments'].unique()
uni

array(['sadness', 'joy', 'fear', 'anger', 'love', 'surprise'],
      dtype=object)

In [100]:
data["Sentiments"]=data.Sentiments.replace({'sadness':0, 'joy':1, 'fear':2, 'anger':3, 'love':4, 'surprise':5})

In [104]:

from tensorflow.keras.losses import CategoricalCrossentropy



In [105]:
X=data["Text"]


In [108]:
from keras.preprocessing.text import Tokenizer

In [112]:
toknaizer=Tokenizer(15212,lower=True,oov_token='UNK')
toknaizer

In [113]:
toknaizer.fit_on_texts(X)

In [125]:
X_train=toknaizer.texts_to_sequences(X)

In [169]:
from keras.preprocessing.sequence import pad_sequences
X_train_paddin=pad_sequences(X_train,maxlen=110,padding='pre')

In [170]:
X_train_paddin

array([[   0,    0,    0, ..., 1276,  104,   59],
       [   0,    0,    0, ...,    2,    3,  620],
       [   0,    0,    0, ...,  621,   19,   64],
       ...,
       [   0,    0,    0, ...,   87, 1100, 3272],
       [   0,    0,    0, ...,  359,  294,   59],
       [   0,    0,    0, ...,  600,   17,  215]], dtype=int32)

In [171]:



y_train=data['Sentiments'].values
# y_train

In [172]:


# convert class vector in to binary class metrics

y_train_cate=to_categorical(y_train)
y_train_cate

array([[1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.]], dtype=float32)

In [173]:
from tensorflow.keras import Sequential

In [179]:

from tensorflow.keras.layers import  Flatten,LSTM,Bidirectional,Embedding,Dense,Dropout
from tensorflow.keras.optimizers import Adam

In [175]:
model=Sequential()

In [176]:
model.add(Embedding(15212,64,input_length=110))
# over fit converting (drop)
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(100,return_sequences=True)))
model.add(Bidirectional(LSTM(160)))
# fully connection layer (dense)
model.add(Dense(6,activation="sigmoid"))

In [178]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_15 (Embedding)    (None, 110, 64)           973568    
                                                                 
 dropout_12 (Dropout)        (None, 110, 64)           0         
                                                                 
 bidirectional_13 (Bidirecti  (None, 110, 200)         132000    
 onal)                                                           
                                                                 
 bidirectional_14 (Bidirecti  (None, 320)              462080    
 onal)                                                           
                                                                 
 dense_1 (Dense)             (None, 6)                 1926      
                                                                 
Total params: 1,569,574
Trainable params: 1,569,574
No

In [180]:
# loss minimising
model.compile(optimizer=Adam(),loss=CategoricalCrossentropy(),metrics=['accuracy'])

In [182]:
model.fit(X_train_paddin,y_train_cate,epochs=9)

Epoch 1/9
31/31 [==============================] - 32s 734ms/step - loss: 1.6048 - accuracy: 0.3086
Epoch 2/9
31/31 [==============================] - 23s 744ms/step - loss: 1.5657 - accuracy: 0.3391
Epoch 3/9
31/31 [==============================] - 22s 725ms/step - loss: 1.4306 - accuracy: 0.4508
Epoch 4/9
31/31 [==============================] - 23s 737ms/step - loss: 0.9956 - accuracy: 0.6365
Epoch 5/9
31/31 [==============================] - 23s 742ms/step - loss: 0.6358 - accuracy: 0.7513
Epoch 6/9
31/31 [==============================] - 24s 761ms/step - loss: 0.4405 - accuracy: 0.8406
Epoch 7/9
31/31 [==============================] - 23s 749ms/step - loss: 0.3573 - accuracy: 0.8832
Epoch 8/9
31/31 [==============================] - 23s 732ms/step - loss: 0.2228 - accuracy: 0.9299
Epoch 9/9
31/31 [==============================] - 23s 750ms/step - loss: 0.1486 - accuracy: 0.9543


In [205]:
def chatbot(word):
  dict={'sadness':0, 'joy':1, 'fear':2, 'anger':3, 'love':4, 'surprise':5}
  for i ,j in dict.items():
    if j==word:
      return i




In [215]:
def prediction(sendence):
  empty_list=[]
  empty_list.append(sendence)
  # text to converting to sequance of intigers
  sendence_toknize=toknaizer.texts_to_sequences(empty_list)
  padded_sequnce=pad_sequences(sendence_toknize,maxlen=110,padding='pre')
  predicted=model.predict(padded_sequnce)
  results=np.argmax(predicted,axis=1)
  result=chatbot(results)
  return result

In [225]:
from IPython.display import clear_output
input_sentence=print("hai what is your name :")
user=input()
input_sentimatal=input("how are you feeling today ")
result=prediction(input_sentimatal)

if result=="sadness" or result=='anger' or result=="fear":
  print("what happend "+user+'?')
  response=input()
  print("hey " +user+ ' am her for you.everyone has a hard time in the life ,"This time will also pass"')
  assistent=print('Do you need any assistance ')
  assistent=input()
  if assistent=='N' or assistent=='n' or assistent=='no' or assistent=='No' or assistent=="NO":
    print('okay , bye '+ user + "have a nice day ")
  elif assistent=='Y' or assistent=='y' or assistent=='yes' or assistent=='Yes' or assistent=="YES":
    print('How can i help you ')
    print('\n')
    print("1 . Do you want to talk with me")
    print('2 . Do you want to listen music')
    response=input()
    if response=='1':
      print("tell me " + user )
      response=input()
      if response=="what is your name" or response=="how are you ":
        print('am chatbot  am good ')


    elif response=='2':
      print('Johny, Johny (Yes, papa)Eating sugar?(No, papa)Telling lies?(No, papa)''\n''Open your mouth(Ah, ah, ah)Johny, Johny(Yes, papa)Eating sugar?(No, papa)''\n''Telling lies?(No, papa)Open your mouth(Ah, ah, ah)')
else:
  print("oh "+user + "your are happy person by friends")



hai what is your name :
abdu
how are you feeling today angry
what happend abdu?
cheat me
hey abdu am her for you.everyone has a hard time in the life ,"This time will also pass"
Do you need any assistance 
yes
How can i help you 


1 . Do you want to talk with me
2 . Do you want to listen music
2
Johny, Johny (Yes, papa)Eating sugar?(No, papa)Telling lies?(No, papa)
Open your mouth(Ah, ah, ah)Johny, Johny(Yes, papa)Eating sugar?(No, papa)
Telling lies?(No, papa)Open your mouth(Ah, ah, ah)
